<a href="https://colab.research.google.com/github/AndreLobo1/pond_mat_2/blob/main/limpeza_tabela_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Carregar a biblioteca pandas

In [1]:
import pandas as pd

Importa o arquivo

In [2]:
file_path = '/content/drive/MyDrive/AUDIENCIA_07_08_2024_COMPLETA.csv'
df = pd.read_csv(file_path)

Verificação de missing values: Não foram encontrados missing values na tabela, portanto, não é necessário realizar nenhum tratamento adicional.








In [3]:
missing_values = df.isnull().sum()
print("Missing values por coluna:")
print(missing_values[missing_values > 0])

Missing values por coluna:
Series([], dtype: int64)


Criar um dicionário para mapear os meses em português para números


In [4]:
meses = {
    'jan.': 1, 'fev.': 2, 'mar.': 3, 'abr.': 4,
    'mai.': 5, 'jun.': 6, 'jul.': 7, 'ago.': 8,
    'set.': 9, 'out.': 10, 'nov.': 11, 'dez.': 12
}

Separar a coluna "PERIODO" em "ANO" e "MÊS"*


In [5]:
df['ANO'] = df['PERIODO'].apply(lambda x: '20' + x.split('-')[1])
df['MÊS'] = df['PERIODO'].apply(lambda x: meses[x.split('-')[0]])

Elimina a coluna 'PERIODO'

In [6]:
df = df.drop(columns=['PERIODO'])

Reordenar as colunas para que "ANO" e "MÊS" sejam as primeiras

In [8]:
colunas = ['ANO', 'MÊS'] + [col for col in df.columns if col not in ['ANO', 'MÊS']]
df = df[colunas]

Verificar tipos de dados das colunas e se existem colunas que precisam ser convertidas


In [9]:
print("Tipos de dados antes da conversão:")
print(df.dtypes)

colunas_para_converter = df.select_dtypes(include=['object']).columns
print("Colunas para converter para numérico:", list(colunas_para_converter))

Tipos de dados antes da conversão:
ANO                   object
MÊS                    int64
TV GAZETA            float64
GAZETA FM VITÓRIA    float64
LITORAL FM           float64
CBN VITORIA          float64
RÁDIO MIX VITÓRIA    float64
INTERNET              object
PORTAL G1/GE/HOME     object
dtype: object
Colunas para converter para numérico: ['ANO', 'INTERNET', 'PORTAL G1/GE/HOME']


Converter colunas 'INTERNET' e 'PORTAL G1/GE/HOME' para numérico: Função para remover separadores de milhar e converter para numérico


In [10]:
def converter_para_numerico(valor):
    if isinstance(valor, str):
        valor = valor.replace('.', '')
        valor = valor.replace(',', '.')
        return pd.to_numeric(valor, errors='coerce')
    return valor

Aplicar a função de conversão nas colunas "INTERNET" e "PORTAL G1/GE/HOME"


In [11]:
df['INTERNET'] = df['INTERNET'].apply(converter_para_numerico)
df['PORTAL G1/GE/HOME'] = df['PORTAL G1/GE/HOME'].apply(converter_para_numerico)

Converter a coluna "ANO" para numérico

In [12]:
df['ANO'] = pd.to_numeric(df['ANO'], errors='coerce')

Identificar colunas categóricas e numéricas

In [13]:
colunas_categoricas = df.select_dtypes(include='object').columns
colunas_numericas = df.drop(colunas_categoricas, axis=1).columns

print(f'Há {len(colunas_categoricas)} Colunas Categóricas: {list(colunas_categoricas)}')
print(f'Há {len(colunas_numericas)} Colunas Numéricas: {list(colunas_numericas)}')


Há 0 Colunas Categóricas: []
Há 9 Colunas Numéricas: ['ANO', 'MÊS', 'TV GAZETA  ', 'GAZETA FM VITÓRIA', 'LITORAL FM', 'CBN VITORIA', 'RÁDIO MIX VITÓRIA', 'INTERNET', 'PORTAL G1/GE/HOME']


Identificar outliers: Os outliers identificados estão corretos, não sendo necessário nenhum tratamento adicional

In [14]:
Q1 = df[colunas_numericas].quantile(0.25)
Q3 = df[colunas_numericas].quantile(0.75)
IQR = Q3 - Q1

outliers = ((df[colunas_numericas] < (Q1 - 1.5 * IQR)) | (df[colunas_numericas] > (Q3 + 1.5 * IQR)))

outlier_counts = outliers.sum()
print("Outliers por coluna:")
print(outlier_counts[outlier_counts > 0])

for coluna in outlier_counts[outlier_counts > 0].index:
    print(f"\nOutliers na coluna '{coluna}':")
    print(df.loc[outliers[coluna], coluna])

Outliers por coluna:
GAZETA FM VITÓRIA    4
INTERNET             1
PORTAL G1/GE/HOME    4
dtype: int64

Outliers na coluna 'GAZETA FM VITÓRIA':
4     8.150
22    7.725
51    7.290
53    7.106
Name: GAZETA FM VITÓRIA, dtype: float64

Outliers na coluna 'INTERNET':
14    10676124
Name: INTERNET, dtype: int64

Outliers na coluna 'PORTAL G1/GE/HOME':
7     4005140
12    4115374
15    4646147
34    5030097
Name: PORTAL G1/GE/HOME, dtype: int64


Arredondar os valores numéricos para 3 casas decimais


In [15]:
df[colunas_numericas] = df[colunas_numericas].round(3)

Exibir as primeiras linhas para verificar a limpeza

In [16]:
print("Primeiras linhas após a reordenação das colunas:")
print(df.head())

Primeiras linhas após a reordenação das colunas:
    ANO  MÊS  TV GAZETA    GAZETA FM VITÓRIA  LITORAL FM  CBN VITORIA  \
0  2020    1      769.843              4.838      18.014        7.144   
1  2020    2      744.739              3.378      18.057        5.576   
2  2020    3      798.373              5.845      22.609        6.969   
3  2020    4      775.029              6.584      23.975        6.532   
4  2020    5      752.017              8.150      23.355        6.290   

   RÁDIO MIX VITÓRIA  INTERNET  PORTAL G1/GE/HOME  
0              1.840   3727780            2372384  
1              1.591   4407044            1496569  
2              2.681   8020768            1787890  
3              3.271   6837563            1711619  
4              2.569   5883559            2010560  
